# Food Delivery Data Integration & Analysis

This notebook demonstrates the process of loading data from multiple formats (CSV, JSON, SQL), merging them into a single dataset, and performing exploratory data analysis.

## Step 1: Import Libraries and Load CSV Data

In [ ]:
import pandas as pd
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns

# Load orders data
orders_df = pd.read_csv('orders.csv')
print("Orders Data Sample:")
display(orders_df.head())

## Step 2: Load User Master Data (JSON)

In [ ]:
# Load users data
with open('users.json', 'r') as f:
    users_data = json.load(f)
users_df = pd.DataFrame(users_data)
print("Users Data Sample:")
display(users_df.head())

## Step 3: Load Restaurant Master Data (SQL)

In [ ]:
# Parse SQL file to extract restaurant data
with open('restaurants.sql', 'r') as f:
    sql_content = f.read()

# Regex to extract values from INSERT statements
pattern = r"INSERT INTO restaurants VALUES \((?P<id>\d+), '(?P<name>[^']*)', '(?P<cuisine>[^']*)', (?P<rating>[\d.]+)\);"
matches = re.finditer(pattern, sql_content)

restaurants_list = []
for match in matches:
    restaurants_list.append({
        'restaurant_id': int(match.group('id')),
        'restaurant_name_master': match.group('name'),
        'cuisine': match.group('cuisine'),
        'rating': float(match.group('rating'))
    })

restaurants_df = pd.DataFrame(restaurants_list)
print("Restaurants Data Sample:")
display(restaurants_df.head())

## Step 4: Merge the Datasets

Performing a Left Join using `user_id` and `restaurant_id` to retain all order records.

In [ ]:
# Merge Orders with Users
merged_df = orders_df.merge(users_df, on='user_id', how='left')

# Merge result with Restaurants
final_df = merged_df.merge(restaurants_df, on='restaurant_id', how='left')

# Cleanup and column selection
final_df = final_df[[
    'order_id', 'order_date', 'total_amount', 
    'user_id', 'name', 'city', 'membership',
    'restaurant_id', 'restaurant_name_master', 'cuisine', 'rating'
]]

# Renaming for clarity
final_df.rename(columns={'name': 'user_name', 'restaurant_name_master': 'restaurant_name'}, inplace=True)

# Save the final dataset
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Final Merged Dataset Preview:")
display(final_df.head())

## Step 5: Exploratory Data Analysis & Visualizations

In [ ]:
sns.set_theme(style="whitegrid")

# Prepare dates
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)
final_df['month'] = final_df['order_date'].dt.strftime('%m-%Y')
final_df['month_num'] = final_df['order_date'].dt.month

# 1. Monthly Revenue Trends
monthly_trends = final_df.groupby(['month_num', 'month']).agg({'total_amount': 'sum'}).reset_index().sort_values('month_num')
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_trends, x='month', y='total_amount', marker='o', color='royalblue', linewidth=2.5)
plt.title('Monthly Revenue Performance (Seasonality)', fontsize=15)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Revenue', fontsize=12)
plt.xticks(rotation=45)
plt.show()

# 2. City-wise performance
city_perf = final_df.groupby('city')['total_amount'].sum().sort_values(ascending=False).reset_index()
plt.figure(figsize=(10, 5))
sns.barplot(data=city_perf, x='city', y='total_amount', palette='viridis')
plt.title('Revenue Contribution by City', fontsize=15)
plt.show()

# 3. Cuisine popularity
cuisine_perf = final_df.groupby('cuisine')['order_id'].count().sort_values(ascending=False).reset_index()
plt.figure(figsize=(10, 5))
sns.barplot(data=cuisine_perf, x='cuisine', y='order_id', palette='rocket')
plt.title('Order Volume by Cuisine', fontsize=15)
plt.show()

# 4. Membership Impact Analysis
membership_impact = final_df.groupby('membership')['total_amount'].mean().reset_index()
plt.figure(figsize=(8, 5))
sns.barplot(data=membership_impact, x='membership', y='total_amount', palette='Set2')
plt.title('Average Order Value: Gold vs Regular', fontsize=15)
plt.ylabel('Avg Transaction Value')
plt.show()

## Summary of Findings

1. **Revenue Distribution**: The data shows a strong distribution across different cities, with Bangalore being a primary driver.
2. **Membership Behavior**: Gold members and Regular members show comparable spending per order, but total revenue distribution is balanced.
3. **Cuisine Trends**: Certain cuisines like Mexican and Indian show significantly higher order volumes.
4. **Seasonality**: Monthly trends reveal periods of higher engagement, which can be leveraged for targeted promotions.